# Get all the data from users on Admin

## Scraping on a site with login

In [1]:
# Prepare a function to read the configuration file with the confidential data...
# ...because the website is an private website on an Intranet, the production file is config.yaml.
# In GitHub there is only a config.yaml.example with dummy data

import yaml

__config = None

def config():

    global __config

    if not __config:
        with open('./scraper_ETL/extract/config.yaml', mode='r') as f:
            config = yaml.safe_load(f)

    return config


In [2]:
# Get the url on the config file
host = config()['website_data']['url']
# host